In [1]:
import sys
sys.path.append('..')

In [2]:
import sentence_embedding_evaluation_german as seeg
import torch
import numpy as np
from torch_emb2vec import ConvToVec
from typing import List

/Users/uh/projects/paper-293/sentence-embedding-evaluation-german/.venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## (1) Instantiate your Embedding model
First, you should load your pretrained embedding.

Here we will generate a random embedding for demonstration purposes.

In [3]:
# generate a random embedding
emb_dim = 256
vocab_sz = 128
emb = torch.randn((vocab_sz, emb_dim), requires_grad=False)
emb = torch.nn.Embedding.from_pretrained(emb)
# assert emb.weight.requires_grad == False

## (2a) Specify the preprocessing
The `preprocessor` function converts a sentences as string into embedding vectors of numbers.

Here we will convert the input strings with a nonsensical approach into IDs for the Embedding layer.

In [4]:
SEQLEN = 192
padid = 128

def preprocesser(batch: List[str], params: dict=None) -> List[List[List[float]]]:
    """ Specify your embedding or pretrained encoder here
    Paramters:
    ----------
    params : dict
        The params dictionary
    batch : List[str]
        A list of sentence as string
    Returns:
    --------
    List[List[List[float]]]
        A list of embedded sequences
    """
    features = []
    for sent in batch:
        # encode
        try:
            ids = torch.tensor([ord(c) % 128 for c in sent])
        except:
            print(sent)
        enc = emb(ids)
        # truncate & pad
        h = torch.ones(size=(SEQLEN, enc.shape[1]), dtype=torch.int64) * padid
        end = min(enc.shape[0], SEQLEN)
        try:
            h[:end, :] = torch.tensor(enc[:end, :])
        except Exception as e:
            raise Exception(e)
        features.append(h)
    features = torch.stack(features, dim=0)
    features = features.type(torch.float32)
    return features.detach()

## (2b) Specify a Customer Classifier

In [5]:
class CustomClassiferModel(torch.nn.Module):
    def __init__(self,
                 embdim: int,
                 output_size: int,  # nclasses
                 *args, **kwargs):
        super(CustomClassiferModel, self).__init__(*args, **kwargs)
        # Self-MHA layer
        self.mha_pre = torch.nn.LayerNorm(embdim, elementwise_affine=True)
        self.mha_net = torch.nn.MultiheadAttention(
            embed_dim=embdim, num_heads=8, batch_first=True, bias=False)
        # 2D to 1D flattening
        self.to1d_pre = torch.nn.LayerNorm(embdim, elementwise_affine=True)
        # compute kernel size and output channel dim
        self.to1d_net = ConvToVec(
            seq_len=SEQLEN, emb_dim=embdim, num_output=384, trainable=True)
        self.to1d_act = torch.nn.GELU()
        # Final layer
        self.final_pre = torch.nn.LayerNorm(384, elementwise_affine=True)
        self.final_net = torch.nn.Linear(384, output_size, bias=False)
        self.final_act = torch.nn.Softmax(dim=1)
        # init params
        self._reset_parameters()

    def _reset_parameters(self) -> None:
        torch.manual_seed(42)
        # Self-MHA projections
        for param in self.mha_net.parameters():
            torch.nn.init.xavier_normal_(param, gain=1.0)
        # Final layer
        torch.nn.init.xavier_normal_(self.final_net.weight, gain=1.0)

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        # Self-MHA layer
        # print("Inputs", inputs.shape)
        h = self.mha_pre(inputs)
        h, _ = self.mha_net(query=h, value=h, key=h)
        h = h + inputs  # skip-conn
        # print("MHA", h.shape)
        # 2D to 1D flattening
        h = self.to1d_pre(h)
        h = self.to1d_net(h)
        h = self.to1d_act(h)
        # print("Flatten", h.shape)
        # Final layer
        h = self.final_pre(h)
        h = self.final_net(h)
        # print("Final", h.shape)
        return self.final_act(h)


def mymodel(**kwargs):
    return CustomClassiferModel(
        embdim=kwargs['n_features'],
        output_size=kwargs['n_classes'])


In [6]:
model = mymodel(n_features=emb_dim, n_classes=3)
n = sum([m.numel() for m in model.parameters()])
print(f"Num params: {n}")

Num params: 364416


## (3) Training settings

In [7]:
params = {
    'datafolder': '../datasets',
    'bias': True,
    'balanced': True,
    'batch_size': 128, 
    'num_epochs': 3,  # Default: 500
    # 'early_stopping': True,
    # 'split_ratio': 0.2,  # if early_stopping=True
    # 'patience': 5,  # if early_stopping=True
}

## (4) Specify downstream tasks

In [8]:
# All
# downstream_tasks = [
#     'TOXIC', 'ENGAGE', 'FCLAIM', 'VMWE',
#     'OL19-A', 'OL19-B', 'OL19-C',
#     'OL18-A', 'OL18-B', 
#     'ABSD-1', 'ABSD-2', 'ABSD-3',
#     'MIO-S', 'MIO-O', 'MIO-I', 'MIO-D', 'MIO-F', 'MIO-P', 'MIO-A',
#     'SBCH-L', 'SBCH-S', 'ARCHI', 'LSDC'
# ]

# Group tasks
# downstream_tasks = [
#     'ABSD-2', 'MIO-S', 'SBCH-S',  # Sentiment analysis
#     'ENGAGE', 'MIO-P',  # engaging/personal
#     'FCLAIM', 'MIO-A',  # fact-claim (potential fake news), argumentative, reasoning
#     'TOXIC', 'OL19-A', 'OL19-B', 'OL19-C', 'MIO-O', 'MIO-I',  # toxic
# ]

# Current favorites
downstream_tasks = ['FCLAIM', 'VMWE', 'OL19-C', 'ABSD-2', 'MIO-P', 'ARCHI', 'LSDC']

## (5) Run experiments

In [ ]:
%%time
results = seeg.evaluate(
    downstream_tasks=downstream_tasks, 
    preprocesser=preprocesser, 
    modelbuilder=mymodel,
    verbose=1,
    **params
)

/Users/uh/projects/paper-293/sentence-embedding-evaluation-german/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Dataset loaded: FCLAIM
epoch 1 | loss: 0.6685386781509106
epoch 2 | loss: 0.5976097973493429
epoch 3 | loss: 0.5702292093863854


/Users/uh/projects/paper-293/sentence-embedding-evaluation-german/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Dataset loaded: VMWE
epoch 1 | loss: 0.7062330807630832
epoch 2 | loss: 0.7167477389940848
epoch 3 | loss: 0.6321420148015022


/Users/uh/projects/paper-293/sentence-embedding-evaluation-german/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Dataset loaded: OL19-C
epoch 1 | loss: 0.7175990007817745
epoch 2 | loss: 0.6447488404810429
epoch 3 | loss: 0.5736871752887964


/Users/uh/projects/paper-293/sentence-embedding-evaluation-german/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Dataset loaded: ABSD-2
epoch 1 | loss: 1.0872514538074796
epoch 2 | loss: 1.0219420496570437
epoch 3 | loss: 0.9905103343097788


/Users/uh/projects/paper-293/sentence-embedding-evaluation-german/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/Users/uh/projects/paper-293/sentence-embedding-evaluation-german/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Dataset loaded: MIO-P


In [ ]:
import json
dat = json.dumps(results, indent=2)
# print(dat)

## (6) Display results

In [ ]:
print("Task | Epochs | N train | N test")
for res in results:
    print(f"{res['task']:>7s}: {res['epochs']:5d} {res['train']['num']:6d} {res['test']['num']:6d}")

In [ ]:
metric = 'acc'  # 'f1', 'f1-balanced', 'acc', 'acc-balanced'
print('  Task | train | test')
for res in results:
    print(f"{res['task']:>7s}: {res['train'][metric]:6.3f} {res['test'][metric]:6.3f}")

In [ ]:
metric = 'f1-balanced'  # 'f1', 'f1-balanced', 'acc', 'acc-balanced'
print('  Task | train | test')
for res in results:
    print(f"{res['task']:>7s}: {res['train'][metric]:6.3f} {res['test'][metric]:6.3f}")

In [ ]:
# class label distributions (inference)
for res in results:
    print(res['task'], res['test']['num'])
    print(res['test']['distr-test'])
    print(res['test']['distr-pred'])

In [ ]:
# class label distribution (training)
for res in results:
    print(res['task'], res['train']['num'])
    print(res['train']['distr-train'])
    print(res['train']['distr-pred'])